In [4]:
import wicked as w
import numpy as np
import re
from fractions import Fraction
from string import Template

In [5]:
def split_single_tensor(tensor):
    """
    Split a single expression or string of the form
    >>> H^{a0,a3}_{a1,a2}
    into a tuple containing its label, the upper, and lower indices, i.e.,
    >>> ('H', [a0, a3], [a1, a2])
    If the label is 'F' or 'V', the upper and lower indices are swapped 
    (see 4c-dsrg-mrpt2.ipynb::dsrg_mrpt2_update to see why)
    """
    tensor = str(tensor)
    label, indices = str(tensor).split('^{')
    upper = indices.split('}_')[0].split(',')
    lower = indices.split('_{')[1].split('}')[0].split(',')
    if label in ['F', 'V']: 
        upper, lower = lower, upper
    return label, upper, lower

def get_unique_tensor_indices(tensor, unused_indices, index_dict):
    """
    >>> ('V', [a0, c0], [a1, a2]) -> 'uiwx'
    Get the indices of a tensor for use in an einsum contraction.
    For example, given the tensor from split_single_tensor, 
    the list of available indices and indices that have already been assigned, 
    we can generate the index string.
    """
    label, upper, lower = tensor
    indstr = ''
    for i in upper+lower:
        if index_dict.get(i):
            indstr += index_dict[i]
        else:
            index = unused_indices[i[0]].pop(0)
            index_dict[i] = index
            indstr += index
    
    return indstr

def get_tensor_slice(tensor, fmt):
    """
    >>> ('V', [a0, c0], [a1, a2]) -> "V['acaa']" (fmt = 'dict')
    >>> ('V', [a0, c0], [a1, a2]) -> "V[a,c,a,a]" (fmt = 'slice')
    """
    if tensor[0] in ['gamma1', 'eta1', 'lambda2', 'lambda3']:
        return tensor[0]
    else:
        if (fmt == 'dict'):
            return tensor[0] + "['" + ''.join([i[0] for i in tensor[1]]) + ''.join([i[0] for i in tensor[2]]) + "']"
        elif (fmt == 'slice'):
            return tensor[0] + '[' + ','.join([i[0] for i in tensor[1]]) + ',' + ','.join([i[0] for i in tensor[2]]) + ']'

def get_lhs_tensor_name(tensor):
    """
    >>> ('V', [a0, c0], [a1, a2]) -> "Vacaa"
    """
    return tensor[0] + ''.join([i[0] for i in tensor[1]]) + ''.join([i[0] for i in tensor[2]])

def get_factor(expression):
    """
    Returns the prefactor of a right hand side expression, taking care of edge cases where an empty space (for +1.0) or a negative sign (for -1.0) are present.
    """
    factor = str(expression).split('+=')[-1].split(' ')[1]
    try:
        return float(Fraction(factor))
    except ValueError:
        if factor == '-':
            return -1.0
        else:
            return 1.0

def compile_einsum(expression, fmt='dict'):
    """
    Compile an expression into a valid einsum expression.
    """
    unused_indices = {'a':list('uvwxyzrst'), 'c':list('ijklmn'), 'v':list('abcdef')}
    index_dict = {}

    lhs = expression.lhs()
    rhs = expression.rhs()

    factor = get_factor(expression)

    exstr = f"+= {factor:+.8f} * np.einsum('"
    tenstr = ''

    for i in str(rhs).split(' '):
        _ = split_single_tensor(i)
        tenstr += get_tensor_slice(_, fmt) + ', '
        exstr += get_unique_tensor_indices(_, unused_indices, index_dict)
        exstr += ','

    exstr = exstr[:-1] + '->'
    tenstr += "optimize='optimal')"

    _ = split_single_tensor(lhs)
    if (_[1] != ['']):
        left = get_lhs_tensor_name(_)
        exstr += get_unique_tensor_indices(_, unused_indices, index_dict)
    else:
        left = _[0]
    exstr += "', " + tenstr

    return left + ' ' + exstr

def make_code(mbeq, fmt='dict'):
    code = ''
    nlines = 0
    for i in mbeq:
        code += compile_einsum(i, fmt) + '\n'
        nlines += 1
    print(f"Generated {nlines} lines of code.\n")
    return code

In [28]:
w.reset_space()
w.add_space("c", "fermion", "occupied", list('ijklmn'))
w.add_space("a", "fermion", "general", list('uvwxyzrst'))
w.add_space("v", "fermion", "unoccupied", list('abcdef'))
wt = w.WickTheorem()

Eref = w.op("E_0",[''])
Fop = w.utils.gen_op('F',1,'cav','cav',diagonal=True)
Vop = w.utils.gen_op('V',2,'cav','cav',diagonal=True)
Hop = Eref + Fop + Vop

T1op = w.utils.gen_op('T1',1,'av','ca',diagonal=False)
T2op = w.utils.gen_op('T2',2,'av','ca',diagonal=False)
Top = T1op + T2op

HT = w.commutator(Hop, Top)
HTexpr = wt.contract(HT, 0, 4)
HTeq = HTexpr.to_manybody_equation("H")

In [29]:
print(make_code(HTeq['c|v'], fmt='dict'))

Generated 48 lines of code.

Hcv += -1.00000000 * np.einsum('ij,ia->ja', F['cc'], T1['cv'], optimize='optimal')
Hcv += -1.00000000 * np.einsum('iu,jiva,vu->ja', F['ca'], T2['ccav'], eta1, optimize='optimal')
Hcv += +1.00000000 * np.einsum('ia,jiba->jb', F['cv'], T2['ccvv'], optimize='optimal')
Hcv += -1.00000000 * np.einsum('ui,va,uv->ia', F['ac'], T1['av'], eta1, optimize='optimal')
Hcv += -1.00000000 * np.einsum('ui,va,uv->ia', F['ac'], T1['av'], gamma1, optimize='optimal')
Hcv += -1.00000000 * np.einsum('uv,iwxa,xv,uw->ia', F['aa'], T2['caav'], eta1, gamma1, optimize='optimal')
Hcv += +1.00000000 * np.einsum('uv,iwxa,uw,xv->ia', F['aa'], T2['caav'], eta1, gamma1, optimize='optimal')
Hcv += +1.00000000 * np.einsum('ua,ivba,uv->ib', F['av'], T2['cavv'], gamma1, optimize='optimal')
Hcv += +1.00000000 * np.einsum('au,iv,vu->ia', F['va'], T1['ca'], eta1, optimize='optimal')
Hcv += +1.00000000 * np.einsum('au,iv,vu->ia', F['va'], T1['ca'], gamma1, optimize='optimal')
Hcv += +1.00000000 * 